In [5]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install sklearn

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import unicodedata
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics

In [ ]:
Este es un proyecto simple que usa Naive Bayes Classifier y Scikit-learn para crear un 
clasificador de reseñas de la tienda Google Play (Análisis de sentimiento) en Python.

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [9]:
df.sample(10)

,package_name,review,polarity
195,com.imangi.templerun2,nice game.... its simply amazing...but i woul...,1
193,com.imangi.templerun2,not supported i am using panasonic t10 and fr...,0
285,com.tencent.mm,if chat at wechat no simbol or bluetick seen...,0
215,com.supercell.clashofclans,sumpreme game but wizards take 8 min and hog...,0
356,com.viber.voip,viber i think so she's going to you guys you ...,0
645,com.uc.browser.en,some problems are solved thank you. feeling g...,1
701,com.opera.mini.native,still the best mini i like the new look and n...,1
235,com.supercell.clashofclans,this game đ.i donno y ppl give it 1 star ...,0
262,com.android.chrome,the best i agree totally with the last poster...,1
28,com.facebook.katana,you should fix this it does'nt work anymore. ...,0


In [13]:
#review va la reseña, vemos las palabras q se repiten mas, tipo bugs, si la encontramos la poneos como cabecera y con cada aparicion le ponemos un 1
#siempre antes primero veo en el texto saco todo lo q es texto ilegible
#limpiamos el data set!
#Eliminamos los caracteres desconocidos

#para hacer NLP limpiamos los textos, dejamos solo texto latino, letras de la A a la Z

#sacamos los espacios vacios al ppio y al final y llevamos todo a minuscula

df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.lower()


In [23]:
def normalize_string(text_string):
    if text_string is not None:
        result = unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result = None
    return result

In [26]:
df['review'] = df['review'].apply(normalize_string)

df['review'] = df['review'].str.replace('!','')
df['review'] = df['review'].str.replace(',','')
df['review'] = df['review'].str.replace('&','')
df['review'] = df['review'].str.normalize('NFKC') #reemplaza texto no latino y trata de arreglar
df['review'] = df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True) #saca tipo loooovvveee y deja love, saca las repeticiones de las letras


#df['review']

In [ ]:
def clean_data(data):
    # Remove package name as it's not relevant
    data = data.drop('package_name', axis=1)
    
    # Convert text to lowercase
    data['review'] = data['review'].str.strip().str.lower()
    return data

#data = clean_data(df)

In [27]:
df.sample(10)

,package_name,review,polarity
363,com.google.android.talk,ok for the simple stuff feature lacking and po...,0
843,com.hamropatro,superb all in one,1
818,com.hamropatro,currency convertor not working it only shows n...,1
21,com.facebook.katana,keeps updating updates all the time slows my p...,0
610,com.evernote,good but ui ease of use still declining i've u...,1
42,com.twitter.android,video issues sometimes videos uploaded from pl...,0
88,com.linkedin.android,better user interface worse quality the user i...,0
495,com.Slack,back to greatness previously couldn't compose ...,1
681,com.hamrokeyboard,daammi aaps dherai ramro lagyo hami sabai nepa...,1
254,com.android.chrome,buggy after update after the update chrome wil...,0


In [28]:
X = df['review']
y = df['polarity']

In [30]:
#train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.25, random_state=42)

In [31]:
# Vectorizo el texto de reviews a numeros 
vector = CountVectorizer(stop_words='english') #le saco las palabras de ingles
X_train = vector.fit_transform(X_train).toarray() #lo convierto en matriz
X_test = vector.transform(X_test).toarray() #

In [32]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [33]:
model.score(X_test, y_test) #score muy bueno

0.8565022421524664

In [36]:
model.predict(vector.transform(['Love']))


array([1])

In [37]:
model.predict(vector.transform(['Bug']))

array([0])

In [40]:
model.predict(vector.transform(['Skype']))

array([0])

In [41]:
#Grabo el modelo
import pickle

filename = '../models/nb_model.sav'
pickle.dump(model, open(filename,'wb'))